<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.定义两个不同的图" data-toc-modified-id="1.定义两个不同的图-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>1.定义两个不同的图</a></span></li><li><span><a href="#2.张量的使用" data-toc-modified-id="2.张量的使用-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>2.张量的使用</a></span></li><li><span><a href="#3.会话的使用" data-toc-modified-id="3.会话的使用-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>3.会话的使用</a></span><ul class="toc-item"><li><span><a href="#3.1-创建和关闭会话" data-toc-modified-id="3.1-创建和关闭会话-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>3.1 创建和关闭会话</a></span></li><li><span><a href="#3.2-使用with-statement-来创建会话" data-toc-modified-id="3.2-使用with-statement-来创建会话-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>3.2 使用with statement 来创建会话</a></span></li><li><span><a href="#3.3-指定默认会话" data-toc-modified-id="3.3-指定默认会话-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>3.3 指定默认会话</a></span></li></ul></li><li><span><a href="#4.使用tf.InteractiveSession构建会话" data-toc-modified-id="4.使用tf.InteractiveSession构建会话-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>4.使用tf.InteractiveSession构建会话</a></span></li><li><span><a href="#5.通过ConfigPrto配置会话" data-toc-modified-id="5.通过ConfigPrto配置会话-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>5.通过ConfigPrto配置会话</a></span></li></ul></div>

# 定义两个不同的图

TensorFlow 默认存在一个计算图(graph)和会话(session);计算图只有加载在会话中，才能进行计算.

In [1]:
import tensorflow as tf

In [2]:
# 清空计算图
tf.reset_default_graph()

# 建立计算图g1
g1 = tf.Graph()
# 把g1设置为默认计算图
with g1.as_default():
    # 申请变量v, 设置初始值为0
    v = tf.get_variable("v", shape=[1], initializer=tf.zeros_initializer())
    
# 建立计算图g2
g2 = tf.Graph()
# 把g2设置为默认计算图
with g2.as_default():
    # 建立变量v，初始值为1
    v = tf.get_variable("v", shape=[1], initializer=tf.ones_initializer())
    
# 建立会话并加载计算图g1
with tf.Session(graph = g1) as sess:
    # 变量初始化
    tf.global_variables_initializer().run()
    
    # 访问变量v
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))
        
# 建立会话并加载计算图g2
with tf.Session(graph = g2) as sess:
    # 变量初始化
    tf.global_variables_initializer().run()
    
    # 访问变量v
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))

[0.]
[1.]


# 张量的使用

In [6]:
# 情况计算图
tf.reset_default_graph()

# 建立张量
a = tf.constant([1.0, 2.0], name="a")
b = tf.constant([2.0, 3.0], name="b")
result = tf.add(a, b)
# 打印Tensor result
print(result)

# 建立会话
sess = tf.InteractiveSession()

# 打印result的结果
print(result.eval())

# 关闭会话
sess.close()

Tensor("Add:0", shape=(2,), dtype=float32)
[ 3.  5.]


# 会话的使用

## 普通方式创建会话

以普通方式创建会话，需要手动关闭会话。

In [7]:
# 创建一个会话
sess = tf.Session()

# 使用会话得到之前的结果
print(sess.run(result))

# 关闭会话是的本次运行中使用的资源可以被释放
sess.close()

[ 3.  5.]


## 使用with statement 来创建会话

In [8]:
with tf.Session() as sess:
    print(sess.run(result))

[ 3.  5.]


## 指定默认会话

In [9]:
sess = tf.Session() # 创建一个会话
with sess.as_default():
    print(result.eval())

[ 3.  5.]


In [11]:
# 建立一个会话
sess = tf.Session()

# 获取张量的数据:有两种方法
print(sess.run(result))
print(result.eval(session=sess))

# 关闭会话
sess.close()

[ 3.  5.]
[ 3.  5.]


## 使用tf.InteractiveSession构建会话

In [12]:
# 构建会话
sess = tf.InteractiveSession()
print(result.eval())
sess.close()

[ 3.  5.]


# 通过ConfigPrto配置会话

通过 ConfigProto 可以配置类似并行的线程数、GPU分配策略、运算超时时间等参数。在这些参数中最常用的有两个:第一个是`allow_soft_placement`，这是一个`bool`类型的参数，当它为`True`时，在以下任意一个条件成立时，GPU上的运算可以放到CPU上进行：
1. 运算无法在GPU上运行；
2. 没有GPU资源(比如运算被指定在第二个GPU上，但是机器上只有一个GPU)；
3. 运算输入包含对CPU计算结果的引用；
这个参数默认为`False`，但是为了使得代码移植性更强，在有GPU的环境下，这个参数一般被设置为`True`。

第二个使用的比较多的配置参数为`log_device_placement`.这也是一个配置参数, 当它为 True时日志中将会记录每个节点被安排在哪个设备上以方便调试。而在生产环境中将这个参数设置为`False`可以减少日志量。

In [3]:
# 配置会话参数
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True # 允许动态分配内存

# 建立会话
sess1 = tf.InteractiveSession(config=config)
sess2 = tf.Session(config=config)

# 关闭会话
sess1.close()
sess2.close()